<a href="https://colab.research.google.com/github/Eshinha/Atividade-III---Sistemas-Multiagentes/blob/main/Atividade_III_Sistemas_Multiagentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentence-transformers hnswlib pandas


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.0 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp311-cp311-lin

In [ ]:
import requests

url = "https://www.gutenberg.org/cache/epub/57001/pg57001.txt"
resposta = requests.get(url)
texto = resposta.text

with open("o_alienista.txt", "w", encoding="utf-8") as f:
    f.write(texto)

print("Texto de O Alienista salvo com sucesso!")


Texto de O Alienista salvo com sucesso!


In [ ]:
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
import hnswlib

with open("o_alienista.txt", encoding="utf-8") as f:
    texto = f.read()

modelo = SentenceTransformer("all-MiniLM-L6-v2")

# Divide por blocos de texto grandes (~500 caracteres cada)
blocos = re.split(r'\n\s*\n', texto)
blocos = [b.strip() for b in blocos if len(b.strip()) > 300]

# Gera embeddings
embeddings = modelo.encode(blocos, convert_to_numpy=True)

# Indexa com hnswlib
index = hnswlib.Index(space="cosine", dim=384)
index.init_index(max_elements=len(blocos), ef_construction=200, M=16)
index.add_items(embeddings)

# Salva
index.save_index("indice.bin")
pd.DataFrame({"bloco": blocos}).to_csv("blocos.csv", index=False)

print(f"{len(blocos)} blocos indexados.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

343 blocos indexados.


Agente A

In [ ]:
import pandas as pd
import hnswlib
from sentence_transformers import SentenceTransformer

modelo = SentenceTransformer("all-MiniLM-L6-v2")
index = hnswlib.Index(space="cosine", dim=384)
index.load_index("indice.bin")
df = pd.read_csv("blocos.csv")

def agenteA_buscar(pergunta):
    emb = modelo.encode([pergunta], convert_to_numpy=True)
    idxs, _ = index.knn_query(emb, k=3)
    return " ".join(df.iloc[i].bloco for i in idxs[0])


Agente D

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def agenteD_resumir(texto):
    if len(texto) > 1000:
        texto = texto[:1000]
    resumo = summarizer(texto, max_length=130, min_length=30, do_sample=False)
    return resumo[0]["summary_text"]


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


Agente B

In [ ]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

def agenteB_responder(pergunta, contexto):
    resposta = qa_pipeline(question=pergunta, context=contexto)
    return resposta["answer"]



Device set to use cpu


Agente C

In [ ]:
def agenteC_avaliar(resposta):
    if len(resposta.split()) < 4:
        return "Resposta curta demais. Reavalie."
    if any(p in resposta.lower() for p in ["não sei", "desconhecido", "irrelevante"]):
        return "Resposta vaga. Solicite nova tentativa."
    return "Resposta satisfatória."


 Executar o sistema

In [ ]:
def executar_fluxo(pergunta):
    print(f"\n[Usuário]: {pergunta}")

    contexto = agenteA_buscar(pergunta)
    print(f"[AgenteA]: Contexto com {len(contexto)} caracteres.")

    resumo = agenteD_resumir(contexto)
    print(f"[AgenteD]: {resumo}")

    resposta = agenteB_responder(pergunta, resumo)
    print(f"[AgenteB]: {resposta}")

    avaliacao = agenteC_avaliar(resposta)
    print(f"[AgenteC]: {avaliacao}")


In [ ]:
executar_fluxo("Por que o personagem Simão Bacamarte decide internar a esposa no hospício?")


[Usuário]: Por que o personagem Simão Bacamarte decide internar a esposa no hospício?
[AgenteA]: Contexto com 1957 caracteres.
[AgenteD]:  Simão Bacamarte officiou aos vereadores, não agrade . Heavio de que o alienista não hadecereceu a parecer tolerante e moderado .
[AgenteB]: não
[AgenteC]: Resposta curta demais. Reavalie.


In [ ]:
executar_fluxo("Como a população de Itaguaí reagiu às decisões de Simão Bacamarte?")


[Usuário]: Como a população de Itaguaí reagiu às decisões de Simão Bacamarte?
[AgenteA]: Contexto com 2455 caracteres.
[AgenteD]:  Simão Bacamarte entendeu reformar tão ruim costume; pediu licença á camara para tratar no edificio that ia construir todos os loucos de Itaguahy and das demais villas e cidades .
[AgenteB]: pediu licença
[AgenteC]: Resposta curta demais. Reavalie.


In [ ]:
executar_fluxo("O que levou Bacamarte a rever sua teoria sobre a loucura?")


[Usuário]: O que levou Bacamarte a rever sua teoria sobre a loucura?
[AgenteA]: Contexto com 1716 caracteres.
[AgenteD]:  D. Evarista achou realmente extraordinarioouslyque toda aquella gente ensandecesse; um ou outro, vá; but todos? Entretanto, custava-lhe duvidar; o marido era um sabio, não recolheria de loucura . A vereança de Itaguahy, entre outros peccados de que é arguida peloschronistas, tinha o de não fazer caso dos dementes .
[AgenteB]: o marido
[AgenteC]: Resposta curta demais. Reavalie.
